# Parsing the data from the PDF files to store in SQL

In [14]:
import PyPDF2
import os
import re
from tqdm import tqdm
import pandas as pd
import sqlite3

In [39]:
def pdf2textReturn(l):
    pdfFileObj = open(l, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    page = pdfReader.getPage(0)
    Text = page.extractText()
    return Text

In [40]:
def PDF2cleanText(path):
    text = []
    DEBUG =False
    directory=path
    l = []
    for file in os.listdir(directory):
        fl = directory + file
        if DEBUG : print (fl)
        l.append(fl)
    l=sorted(l)
    for i in l:
        text.append(pdf2textReturn(i))
    return text

In [41]:
# path of the directory which holds the pdf files
path = '/home/reynold/Desktop/Zauba/data/'

In [42]:
hold=[]
rtt = PDF2cleanText(path)
for i in tqdm(range(len(rtt))):
    Text = rtt[i] 
    SRN  = re.findall('([U]\d{1,8})',Text)[0]
    SRD  = re.findall('(\d\d/\d\d/\d\d\d\d)',Text)[0]
    SRA  = re.findall(r':Address.+\d\d\d\d\d\d\d\d',Text)[0].split(":")[2]
    STYP = re.findall(r'Type:.+Service',Text)[0].split(":")[1]
    SDES = re.sub(r'Remittance','',re.findall(r'Remittance.+\d\d\d\d',Text)[0])+')'
    SRPS = re.sub(r'Received Payment Rupees:','',re.findall(r'Received Payment Rupees:.+',Text)[0])
    SMOD = re.sub(r'Mode of Payment:Note:','',re.findall(r'Mode of Payment:.+Received',Text)[0])
    STOT = re.sub(r'Total','',re.findall(r'Total\S+',Text)[0])
    pat  = re.findall(r'\S+.\d\d.Total',Text)[0].split(")")[1].split()[0]
    SAMT = re.findall(r'\d\S+',pat)[0]
    SFEE = re.sub(SAMT,'',pat)
    hold.append([SRN,SRD,SRA,STYP,SDES,SRPS,STOT,SAMT,SFEE])

100%|██████████| 3/3 [00:00<00:00, 3048.92it/s]


In [43]:
SRN  = "SRN_no"
SRD  = "Date"
SRA  = "Name_and_Address"
STYP = "Service_Type"
SDES = "Service_Description"
SRPS = "Received_Payment_Rupees"
STOT = "Total"
SAMT = "Amount"
SFEE = "Types_of_Fees"

colname = ["SRN_no","Date","Name_and_Address","Service_Type","Service_Description",
          "Received_Payment_Rupees","Total","Amount","Types_of_Fees"]

In [44]:
# Creating a dataframe for storing the pdf data
dataframe = pd.DataFrame(hold,columns=colname)

In [45]:
dataframe

,SRN_no,Date,Name_and_Address,Service_Type,Service_Description,Received_Payment_Rupees,Total,Amount,Types_of_Fees
0,U16571275,03/08/2017,Zauba Technologies and Data Services PrivatNo ...,Fee for inspection of Public documentsService,Inspection of Public documents of KEYSTONE REA...,One Hundred Only,100.00,100.00,Normal
1,U16572745,03/08/2017,Zauba Technologies and Data Services PrivatNo ...,Fee for inspection of Public documentsService,Inspection of Public documents of LANDMARK CRA...,One Hundred Only,100.00,100.00,Normal
2,U16573131,03/08/2017,Zauba Technologies and Data Services PrivatNo ...,Fee for inspection of Public documentsService,Inspection of Public documents of WESNIA INFO ...,One Hundred Only,100.00,100.00,Normal


In [52]:
#Zauba is the database name 
conn = sqlite3.connect("Zauba.db")
cur = conn.cursor()

In [53]:
# Receipt is the name of the table in the Zauba database
# if a table already exists and needs to be updated the parameter if_exists is to be changed to append
dataframe.to_sql("receipt", conn, if_exists="replace")


# Readng the data from sql database

In [54]:
pd.read_sql_query("select * from receipt limit 2;", conn)

,index,SRN_no,Date,Name_and_Address,Service_Type,Service_Description,Received_Payment_Rupees,Total,Amount,Types_of_Fees
0,0,U16571275,03/08/2017,Zauba Technologies and Data Services PrivatNo ...,Fee for inspection of Public documentsService,Inspection of Public documents of KEYSTONE REA...,One Hundred Only,100.00,100.00,Normal
1,1,U16572745,03/08/2017,Zauba Technologies and Data Services PrivatNo ...,Fee for inspection of Public documentsService,Inspection of Public documents of LANDMARK CRA...,One Hundred Only,100.00,100.00,Normal


In [25]:
conn.close()